In [14]:
import pandas as pd
import numpy as np

from datetime import datetime, timezone
import ollama

import sys

sys.path.append("..")

# dlt imports
import dlt
from rest_api import RESTAPIConfig, rest_api_source

from dlt.sources.helpers.rest_client.paginators import (
    BasePaginator,
    JSONResponsePaginator,
)
from dlt.sources.helpers.requests import Response, Request

from dlt.destinations.adapters import lancedb_adapter

import lancedb

# Homework

In the workshop, we extracted contents from two pages in notion titled "Workshop: Benefits and Perks" and "Workshop: Working hours, PTO, and Vacation". 

Repeat the same process for a third page titled "Homework: Employee handbook" (hidden from public view, but accessible via API key):

1. Modify the REST API source to extract only this page.
2. Write the output into a separate table called "homework".
3. Remember to update the table name in all cells where you connect to a lancedb table.

To do this you can use the [workshop Colab](https://colab.research.google.com/drive/1nNOybHdWQiwUUuJFZu__xvJxL_ADU3xl?usp=sharing) as a basis.

Now, answer the following questions:

## Answer
 
First, we need to install both `dlt` (with `lancedb` addons) and `sentence-transformers` (this should be already installed). Execute this pip install

```bash
pip install dlt[lancedb]==0.5.1a0
```

Then, we need to create a **`dlt` project**, so we execute this line

```bash
yes | dlt init rest_api lancedb
```

This will create several files and directories in our root path (if executed from a notebook, move inside the notebook directory):
- directory `.dlt` will be created, which contains the configuration files used by `dlt`
    - File `./.dlt/secrets.toml` can be used to configure access to APIs, as well as to store any secrets.
- directory `.lancedb`, which contains the data files used by the database
- module `rest_api`, which contains neccesary functions to be used by dlt when connecting to an API

We have to configure `secrets.toml`:

```yaml
[sources.rest_api.notion]
api_key = "api_key" # please set me up

[destination.lancedb]
embedding_model_provider = "sentence-transformers"
embedding_model = "all-MiniLM-L6-v2"

[destination.lancedb.credentials]
uri = ".lancedb"
api_key = "api_key"
embedding_model_provider_api_key = "embedding_model_provider_api_key"
```


Once this is done, we need to create the pipeline code


In [2]:
# if dlt is not able to find `secrets.toml`, execute this
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["SOURCES__REST_API__NOTION__API_KEY"] = os.environ.get(
    "SOURCES__REST_API__NOTION__API_KEY"
)

os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

os.environ["DESTINATION__LANCEDB__CREDENTIALS__URI"] = ".lancedb"

In [12]:
class PostBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None

    def update_state(self, response: Response) -> None:
        # Assuming the API returns an empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False

    def update_request(self, request: Request) -> None:
        if request.json is None:
            request.json = {}

        # Add the cursor to the request body
        request.json["start_cursor"] = self.cursor


@dlt.resource(name="employee_handbook")
def rest_api_notion_resource():
    notion_config: RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {"token": dlt.secrets["sources.rest_api.notion.api_key"]},
            "headers": {
                "Content-Type": "application/json",
                "Notion-Version": "2022-06-28",
            },
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "homework",  # changed to "homework" (from "workshop") to search only pages that contains "homework"
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time",
                        },
                    },
                    "data_selector": "results",
                },
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id",
                        }
                    },
                },
            },
        ],
    }

    yield from rest_api_source(notion_config, name="employee_handbook")


def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content,
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc),
    }


@dlt.resource(
    name="employee_handbook",
    write_disposition="merge",
    primary_key="block_id",
    columns={"last_edited_time": {"dedup_sort": "desc"}},
)
def rest_api_notion_incremental(
    last_edited_time=dlt.sources.incremental(
        "last_edited_time",
        initial_value="2024-06-26T08:16:00.000Z",
        primary_key=("block_id"),
    )
):
    last_value = last_edited_time.last_value
    print(last_value)

    for block in rest_api_notion_resource.add_map(extract_page_content):
        if not (len(block["content"])):
            continue
        yield block


def load_notion() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="company_policies",
        destination="lancedb",
        dataset_name="notion_pages",
        # full_refresh=True
    )

    load_info = pipeline.run(
        lancedb_adapter(rest_api_notion_incremental, embed="content"),
        table_name="homework",  # changed from "employee_handbook" (we want to create a different table)
        write_disposition="merge",
    )
    print(load_info)


load_notion()

2024-07-05T23:33:00.000Z
Pipeline company_policies load step completed in ---
0 load package(s) were loaded to destination LanceDB and into dataset None
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x7a7685c73df0> location to store data


Now, if we go to `.lancedb` we find directories `notion_pages___dlt_loads....` which contains the retrieved data.

Let's visualize the data:

In [5]:
db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___homework")

dbtable.to_pandas()

,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,c69f1ecf-7b02-5810-8286-3f42659ae9d4,"[-0.024265617, 0.04746088, -0.011796454, 0.063...",a8196881-ae94-4767-8767-92fe1a327d24,paragraph,We owe our success to our employees. To show o...,2024-07-05 22:34:00+00:00,2024-07-21 19:13:59.992439+00:00,1721589238.948023,MCLoVTAl0R81VA
1,f2c18ac0-50f5-5b72-a871-dc5a46780353,"[-0.049661588, 0.108535156, -0.0097626075, -0....",31fcbf26-2ca5-468a-8af8-d7eb4c2db8c8,paragraph,We want to ensure that private information abo...,2024-07-05 22:38:00+00:00,2024-07-21 19:13:59.995691+00:00,1721589238.948023,p3uGFEoILLOozg
2,4553193e-c655-54df-9a33-cfc570bf34d0,"[-0.06316318, 0.17331506, 0.025351683, -0.0191...",da7721fd-3d0f-4c04-bc5e-825ad60bed1c,paragraph,Employee health is important to us. We don’t d...,2024-07-05 22:52:00+00:00,2024-07-21 19:13:59.995866+00:00,1721589238.948023,rgP02jz1tOdhoA
3,791be1a1-6c67-530d-87ab-bd9912500ea5,"[-0.10974314, 0.10586077, 0.0032905845, -0.021...",ff36dcf3-5faa-40b4-ad8e-92fdc952201e,paragraph,Our company is dedicated to maintaining a safe...,2024-07-05 22:52:00+00:00,2024-07-21 19:13:59.995975+00:00,1721589238.948023,MHJzxOmwDxORQA
4,a83497f4-922c-5d62-bab1-53804e93c811,"[0.052423306, -0.064575925, 0.06586298, 0.0145...",a1ff9697-4bb6-4f1e-b464-dda296dbd307,paragraph,If your job doesn’t require you to be present ...,2024-07-05 22:52:00+00:00,2024-07-21 19:13:59.996085+00:00,1721589238.948023,17E8UdgbSNcpsA
5,434b71e9-a11a-519d-a9fe-e3ade78d47d6,"[0.0005233398, -0.05488338, 0.04357337, -0.010...",e4ec9f4d-b687-4c28-a80d-985bfabcc2ba,paragraph,Remote working refers to working from a non-of...,2024-07-05 22:52:00+00:00,2024-07-21 19:13:59.996193+00:00,1721589238.948023,g07i9/k6wp5ohQ
6,17816363-54b7-5ba7-b8d5-06d871a25414,"[0.03802626, -0.021509647, 0.04752782, 0.06470...",e6e550dc-b59e-4928-abd7-07eace948681,paragraph,There are some expenses that we will pay direc...,2024-07-05 22:52:00+00:00,2024-07-21 19:13:59.996302+00:00,1721589238.948023,/kTI+w8snNC0Yg
7,2a434cf9-09d9-5514-a88b-02977f2f953e,"[-0.05858809, -0.075404465, 0.033775214, 0.009...",a269d0ca-ce14-481b-a5f4-9192d6840d6e,paragraph,Our company operates between 9 a.m. to 7 p.m. ...,2024-07-05 22:52:00+00:00,2024-07-21 19:13:59.996412+00:00,1721589238.948023,21AeARULeUEIdA
8,5f9384fa-7f98-5f52-a06e-05b05f42f69a,"[-0.013599298, 0.004753062, 0.02483512, 0.0159...",5b65f3e7-0a37-429a-818d-f99b53755ebd,paragraph,"In this section, we are going to be covering i...",2024-07-05 23:33:00+00:00,2024-07-21 19:13:59.996526+00:00,1721589238.948023,SbmPDPC1o1FWsQ
9,42af72f6-9db7-54a2-87b2-d466169078ff,"[0.03206086, 0.024244644, 0.008471348, 0.03179...",b27f7d80-f2f1-460e-aa0c-b8e770cf050a,paragraph,Our company observes the following holidays: N...,2024-07-05 22:52:00+00:00,2024-07-21 19:13:59.996627+00:00,1721589238.948023,gNPkUaSmua8azA




## Q1. Rows in LanceDB

How many rows does the lancedb table "notion_pages__homework" have?

* 14
* 15
* 16
* 17

## Answer

Let's visualize the data that we have just donwloaded into `lancedb`

In [6]:
db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___homework")

dbtable.to_pandas()

,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,c69f1ecf-7b02-5810-8286-3f42659ae9d4,"[-0.024265617, 0.04746088, -0.011796454, 0.063...",a8196881-ae94-4767-8767-92fe1a327d24,paragraph,We owe our success to our employees. To show o...,2024-07-05 22:34:00+00:00,2024-07-21 19:13:59.992439+00:00,1721589238.948023,MCLoVTAl0R81VA
1,f2c18ac0-50f5-5b72-a871-dc5a46780353,"[-0.049661588, 0.108535156, -0.0097626075, -0....",31fcbf26-2ca5-468a-8af8-d7eb4c2db8c8,paragraph,We want to ensure that private information abo...,2024-07-05 22:38:00+00:00,2024-07-21 19:13:59.995691+00:00,1721589238.948023,p3uGFEoILLOozg
2,4553193e-c655-54df-9a33-cfc570bf34d0,"[-0.06316318, 0.17331506, 0.025351683, -0.0191...",da7721fd-3d0f-4c04-bc5e-825ad60bed1c,paragraph,Employee health is important to us. We don’t d...,2024-07-05 22:52:00+00:00,2024-07-21 19:13:59.995866+00:00,1721589238.948023,rgP02jz1tOdhoA
3,791be1a1-6c67-530d-87ab-bd9912500ea5,"[-0.10974314, 0.10586077, 0.0032905845, -0.021...",ff36dcf3-5faa-40b4-ad8e-92fdc952201e,paragraph,Our company is dedicated to maintaining a safe...,2024-07-05 22:52:00+00:00,2024-07-21 19:13:59.995975+00:00,1721589238.948023,MHJzxOmwDxORQA
4,a83497f4-922c-5d62-bab1-53804e93c811,"[0.052423306, -0.064575925, 0.06586298, 0.0145...",a1ff9697-4bb6-4f1e-b464-dda296dbd307,paragraph,If your job doesn’t require you to be present ...,2024-07-05 22:52:00+00:00,2024-07-21 19:13:59.996085+00:00,1721589238.948023,17E8UdgbSNcpsA
5,434b71e9-a11a-519d-a9fe-e3ade78d47d6,"[0.0005233398, -0.05488338, 0.04357337, -0.010...",e4ec9f4d-b687-4c28-a80d-985bfabcc2ba,paragraph,Remote working refers to working from a non-of...,2024-07-05 22:52:00+00:00,2024-07-21 19:13:59.996193+00:00,1721589238.948023,g07i9/k6wp5ohQ
6,17816363-54b7-5ba7-b8d5-06d871a25414,"[0.03802626, -0.021509647, 0.04752782, 0.06470...",e6e550dc-b59e-4928-abd7-07eace948681,paragraph,There are some expenses that we will pay direc...,2024-07-05 22:52:00+00:00,2024-07-21 19:13:59.996302+00:00,1721589238.948023,/kTI+w8snNC0Yg
7,2a434cf9-09d9-5514-a88b-02977f2f953e,"[-0.05858809, -0.075404465, 0.033775214, 0.009...",a269d0ca-ce14-481b-a5f4-9192d6840d6e,paragraph,Our company operates between 9 a.m. to 7 p.m. ...,2024-07-05 22:52:00+00:00,2024-07-21 19:13:59.996412+00:00,1721589238.948023,21AeARULeUEIdA
8,5f9384fa-7f98-5f52-a06e-05b05f42f69a,"[-0.013599298, 0.004753062, 0.02483512, 0.0159...",5b65f3e7-0a37-429a-818d-f99b53755ebd,paragraph,"In this section, we are going to be covering i...",2024-07-05 23:33:00+00:00,2024-07-21 19:13:59.996526+00:00,1721589238.948023,SbmPDPC1o1FWsQ
9,42af72f6-9db7-54a2-87b2-d466169078ff,"[0.03206086, 0.024244644, 0.008471348, 0.03179...",b27f7d80-f2f1-460e-aa0c-b8e770cf050a,paragraph,Our company observes the following holidays: N...,2024-07-05 22:52:00+00:00,2024-07-21 19:13:59.996627+00:00,1721589238.948023,gNPkUaSmua8azA


In [9]:
print(dbtable.to_pandas().iloc[-1, :]["content"])

If you want to invite a visitor to our offices, please ask for permission from our HR Manager first. Also, inform our reception of your visitor’s arrival. Visitors should sign in and show identification. They will receive passes and will be asked to return them to reception once their visit is complete. When you have office visitors, you also have responsibilities. You should: Always tend to your visitors (especially when they are underage.), keep your visitors away from areas where there are dangerous machines, chemicals, confidential records or sensitive equipment. Prevent your visitors from proselytizing your colleagues, gathering donations or requesting participation in activities while on our premises. Anyone who delivers orders, mail or packages for employees should remain at our building’s reception or gate. If you are expecting a delivery, [front office employees/ security guards] will notify you so you may collect it.


In [10]:
dbtable.to_pandas().shape

(17, 9)

There are 17 rows

## Q2. Running the Pipeline: Last edited time

In the demo, we created an incremental dlt resource `rest_api_notion_incremental` that keeps track of `last_edited_time`. What value does it store after you've run your pipeline once? (Hint: you will be able to get this value by performing some aggregation function on the column `last_edited_time` of the table)

* `Timestamp('2024-07-05 22:34:00+0000', tz='UTC') (OR "2024-07-05T22:34:00.000Z")`
* `Timestamp('2024-07-05 23:33:00+0000', tz='UTC') (OR "2024-07-05T23:33:00.000Z")`
* `Timestamp('2024-07-05 23:52:00+0000', tz='UTC') (OR "2024-07-05T23:52:00.000Z")`
* `Timestamp('2024-07-05 22:56:00+0000', tz='UTC') (OR "2024-07-05T22:56:00.000Z")`

## Answer

The value of `last_edited_time` is equal to `2024-07-05T23:33:00.000Z`

In [13]:
dbtable.to_pandas()["last_edited_time"].max()

Timestamp('2024-07-05 23:33:00+0000', tz='UTC')

## Q3. Ask the Assistant 

Find out with the help of the AI assistant: how many PTO days are the employees entitled to in a year?  

* 20
* 25
* 30
* 35

## Answer

First we need to run `ollama` locally

```bash
docker run -it \
    --rm \
    -v ollama:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama
```

Then we have to pull the model:

```bash
docker exec -it ollama bash
ollama pull llama2-uncensored
```

Then, we are going to use the python library `ollama` to interact with ollama (instead of using OpenAI's library as we did in the last modules of the course)

```bash
pip install ollama
```

In [15]:
def retrieve_context_from_lancedb(dbtable, question, top_k=2):

    query_results = dbtable.search(query=question).to_list()
    context = "\n".join([result["content"] for result in query_results[:top_k]])

    return context

In [16]:
def main(question: str = "You: "):
    # Connect to the lancedb table
    db = lancedb.connect(".lancedb")
    dbtable = db.open_table("notion_pages___homework")

    # A system prompt telling ollama to accept input in the form of "Question: ... ; Context: ..."
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant that helps users understand policies inside a company's employee handbook. The user will first ask you a question and then provide you relevant paragraphs from the handbook as context. Please answer the question based on the provided context. For any details missing in the paragraph, encourage the employee to contact the HR for that information. Please keep the responses conversational.",
        }
    ]

    # while True:
    # Accept user question
    # question = input("You: ")

    # Retrieve the relevant paragraphs on the question
    context = retrieve_context_from_lancedb(dbtable, question, top_k=2)

    # Create a user prompt using the question and retrieved context
    messages.append(
        {"role": "user", "content": f"Question: '{question}'; Context:'{context}'"}
    )

    # Get the response from the LLM
    response = ollama.chat(model="llama2-uncensored", messages=messages)
    response_content = response["message"]["content"]
    print(f"Assistant: {response_content}")

    # Add the response into the context window
    messages.append({"role": "assistant", "content": response_content})

    return messages

In [17]:
question = "how many PTO days are the employees entitled to in a year?"
response = main(question=question)
print(response)

Assistant: Sure! I can definitely help with that! Based on the context provided, it seems that employees are entitled to 30 days of paid time off (PTO) per year. This includes sick leave, vacation, and holidays. However, if you'd like more specific information about your own employee benefits package or PTO usage, I recommend speaking with HR directly or reviewing the company's policies in detail.
[{'role': 'system', 'content': "You are a helpful assistant that helps users understand policies inside a company's employee handbook. The user will first ask you a question and then provide you relevant paragraphs from the handbook as context. Please answer the question based on the provided context. For any details missing in the paragraph, encourage the employee to contact the HR for that information. Please keep the responses conversational."}, {'role': 'user', 'content': "Question: 'how many PTO days are the employees entitled to in a year?'; Context:'In this section, we are going to be 

Employees are entitled to 30 days of paid time off (PTO)

## Submit the results

* Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2024/homework/workshop1
* It's possible that your answers won't match exactly. If it's the case, select the closest one.